In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [ ]:
titles=list(df.columns)
titles[1], titles[2]=titles[2], titles[1]
df=df[titles]
df.head()

,id,comment_text,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape, val_df.shape

((1714630, 45), (90244, 45))

In [ ]:
train_toxic = train_df[train_df['target']>=0.5]
train_clean = train_df[train_df['target']< 0.5]
print(len(train_toxic))
print(len(train_clean))
val_toxic = val_df[val_df['target']>=0.5]
val_clean = val_df[val_df['target']< 0.5]
print(len(val_toxic))
print(len(val_clean))

137075
1577555
7259
82985


In [ ]:
train_df = pd.concat([
  train_toxic.sample(10000),
  train_clean.sample(10000)
])
val_df = pd.concat([
  val_toxic.sample(1000),
  val_clean.sample(1000)
])
train_df.shape, val_df.shape

((1000, 45), (200, 45))

In [ ]:
target_cols=train_df.columns.tolist()[2:8]
target_cols

['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(list(train_df["comment_text"]), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df["comment_text"]), truncation=True, padding=True)

In [ ]:
import torch

class JigsawDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JigsawDataset(train_encodings, train_df.iloc[:,2:8].values)
val_dataset = JigsawDataset(val_encodings, val_df.iloc[:,2:8].values)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import BertForSequenceClassification
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
import torch.nn as nn

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6, output_attentions=False, output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=4e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

for epoch in range(100):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs, labels=labels)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Validation accuracy - epoch_{epoch}: {accuracy:.4f}")
    print(f"Precision score - epoch_{epoch}: {precision:.4f}")
    print(f"Recall score - epoch_{epoch}: {recall:.4f}")
    print(f"F1 score - epoch_{epoch}: {f1:.4f}")

Validation accuracy - epoch_0: 0.9500
Precision score - epoch_0: 0.4774
Recall score - epoch_0: 0.9500
F1 score - epoch_0: 0.9305
Validation accuracy - epoch_1: 0.9550
Precision score - epoch_1: 0.9775
Recall score - epoch_1: 0.9550
F1 score - epoch_1: 0.9330
Validation accuracy - epoch_2: 0.9500
Precision score - epoch_2: 0.6464
Recall score - epoch_2: 0.9500
F1 score - epoch_2: 0.9379


In [ ]:
from scipy.stats import ttest_rel

accuracy_run1 = [0.85, 0.88, 0.82]
precision_run1 = [0.84, 0.87, 0.81]
recall_run1 = [0.86, 0.89, 0.83]
f1_run1 = [0.82, 0.85, 0.80]

accuracy_run2 = [0.88, 0.90, 0.87]
precision_run2 = [0.87, 0.89, 0.86]
recall_run2 = [0.89, 0.91, 0.88]
f1_run2 = [0.85, 0.88, 0.84]

t_stat_accuracy, p_value_accuracy = ttest_rel(accuracy_run1, accuracy_run2)

t_stat_precision, p_value_precision = ttest_rel(precision_run1, precision_run2)

t_stat_recall, p_value_recall = ttest_rel(recall_run1, recall_run2)

t_stat_f1, p_value_f1 = ttest_rel(f1_run1, f1_run2)

print(f"Accuracy t-statistic: {t_stat_accuracy}, p-value: {p_value_accuracy}")
print(f"Precision t-statistic: {t_stat_precision}, p-value: {p_value_precision}")
print(f"Recall t-statistic: {t_stat_recall}, p-value: {p_value_recall}")
print(f"F1 t-statistic: {t_stat_f1}, p-value: {p_value_f1}")


Accuracy t-statistic: -3.7796447300922718, p-value: 0.06341418841830604
Precision t-statistic: -3.7796447300922824, p-value: 0.06341418841830572
Recall t-statistic: -3.7796447300922718, p-value: 0.06341418841830604
F1 t-statistic: -10.0000000000001, p-value: 0.009852457023325498


In [ ]:
text = "I will kill you"
encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.logits

probabilities = torch.sigmoid(logits)

print(probabilities)

tensor([[0.4273, 0.0165, 0.0534, 0.0244, 0.3642, 0.0251]], device='cuda:0')


In [ ]:
probabilities = probabilities.cpu()
array = probabilities.detach().numpy()
array

array([[0.427347  , 0.0165028 , 0.05340241, 0.02438422, 0.36417362,
        0.02514137]], dtype=float32)

In [ ]:
array=array.flatten()

In [ ]:
for label, prediction in zip(target_cols, array):
  print(f"{label}: {prediction}")

target: 0.42734700441360474
severe_toxicity: 0.01650279574096203
obscene: 0.05340241268277168
identity_attack: 0.02438422478735447
insult: 0.36417362093925476
threat: 0.02514137327671051
